# Saliva Data Processing

## Setup and Helper Functions

In [ ]:
import re
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

import biopsykit as bp

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
plt.close("all")

palette = bp.colors.fau_palette
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.default'] = "regular"

palette

## Data Import

In [ ]:
data_path = Path("../../../../Data")
# directories within the Data repository
saliva_path = data_path.joinpath("saliva")
saliva_path_import = saliva_path.joinpath("raw")
# path to export processed saliva data in the Data repository
saliva_path_export = saliva_path.joinpath("processed")
# path to export processed saliva data in the Analysis repository
saliva_path_export_analysis = Path("../../../data/processed/saliva")

bp.utils.file_handling.mkdirs([saliva_path_export, saliva_path_export_analysis])

In [ ]:
condition_list = bp.io.load_subject_condition_list(data_path.joinpath("condition_list.csv"))
condition_list.head()

In [ ]:
data = bp.io.saliva.load_saliva_plate(saliva_path_import.joinpath("cft_cortisol_data.xlsx"), saliva_type="cortisol", condition_list=condition_list)
data.head()

## Data Processing

In [ ]:
sample_times = [-30, -1, 30, 40, 50, 60, 70]

In [ ]:
auc = bp.saliva.auc(data, remove_s0=True, sample_times=sample_times, compute_auc_post=True)

In [ ]:
max_inc = bp.saliva.max_increase(data, remove_s0=True)
max_inc = max_inc.join(bp.saliva.max_increase(data, remove_s0=True, percent=True))

In [ ]:
slope = bp.saliva.slope(data, sample_labels=["S1", "S4"], sample_times=sample_times)

In [ ]:
cort_features = pd.concat([auc, max_inc, slope], axis=1)
cort_features = bp.saliva.utils.saliva_feature_wide_to_long(cort_features, saliva_type="cortisol")
cort_features.head()

## Export

In [ ]:
data.to_csv(saliva_path_export.joinpath("cortisol_samples.csv"))
data.to_csv(saliva_path_export_analysis.joinpath("cortisol_samples.csv"))

In [ ]:
cort_features.to_csv(saliva_path_export.joinpath("cortisol_features.csv"))
cort_features.to_csv(saliva_path_export_analysis.joinpath("cortisol_features.csv"))